# Journal de Bord - Projet IA Prediction Reussite Scolaire

## Jour 1 - 11/01/2026

### Étapes / Actions réalisées
- Initialisation de l'espace de travail.
- Création des dossiers `FINAL/notebooks` pour isoler le travail des sources.
- Copie des templates `matrice-notebook.ipynb` et `journal-de-bord.ipynb`.
- Chargement et exploration initiale des données (`student-mat.csv`, `student-por.csv`).
- Vérification de la structure des fichiers (séparateur `;`).

### Observations / Difficultés
- Pas de valeurs manquantes détectées.
- Distribution de G3 assez normale mais avec des 0 (abandons ?).
- Forte corrélation entre G1, G2 et G3 confirmée.

## Jour 2 - 12/01/2026

### Étapes / Actions réalisées
- Mise en place du pipeline de preprocessing : `OneHotEncoder` pour les catégories, `StandardScaler` pour les numériques.
- Création de la fonction `train_evaluate` pour standardiser les tests.
- Test des modèles `LinearRegression` et `RandomForestRegressor`.

### Observations / Difficultés
- Random Forest performe mieux que la Régression Linéaire.
- Les scores R2 sont très élevés (> 0.8) car G1 et G2 sont inclus (Scénario "Final").

## Jour 3 - 13/01/2026

### Étapes / Actions réalisées
- Implémentation du scénario "Prédiction Précoce" (sans G1 ni G2).
- Visualisation de l'importance des variables pour Random Forest.
- Comparaison des résultats :
    - **Avec G1/G2** : R2 ~0.81 (RW), ~0.84 (Por)
    - **Sans G1/G2** : R2 ~0.24 (Math), ~0.17 (Por)

### Observations / Difficultés
- La performance chute drastiquement sans les notes intermédiaires (G1, G2).
- Prédire la note finale (G3) uniquement sur des critères socio-démographiques est difficile avec ces modèles.
- Les variables importantes deviennent : `failures`, `absences`, `Medu`, `Fedu`.

### Prochaines étapes
- Fusionner les datasets pour analyser les élèves communs.
- Feature Engineering pour tenter d'améliorer le scénario "Précoce".

## Jour 4 - 15/01/2026

### Étapes / Actions réalisées
- Fusion des datasets Math et Portugais basée sur les attributs communs.
- Vérification du nombre d'étudiants communs : **382**.
- Analyse de la corrélation entre la réussite en Math (`G3_mat`) et en Portugais (`G3_por`) : **0.48**.
- **Feature Engineering** :
    - Création de `TotalAlc` (Dalc + Walc) : *Justifié par une corrélation forte (0.65) entre semaine/weekend et un impact identique sur G3 (-0.05)*.
    - Création de `ParentEdu` (Medu + Fedu), `HasFailed` (failures > 0).
    - Ré-évaluation du scénario "Prédiction Précoce" avec ces nouvelles features.
- **Scénario 3 (Merge/Combine)** :
    - Entrainement sur un jeu de données combiné (Math + Por, N=1044) avec ajout d'une feature `Subject`.

### Observations / Difficultés
- **Feature Engineering** :
    - Impact limité. Math R2 monte légèrement à **0.259**.
- **Dataset Combiné** :
    - Random Forest R2 = **0.241**. RMSE = 3.426.
    - L'augmentation du volume de données ne compense pas la difficulté intrinsèque de prédire G3 sans G1/G2.
    - Cela confirme que la réussite scolaire dépend fortement de facteurs temporels (notes précédentes) ou non capturés par ce dataset (psychologie, contexte classe, etc.).

### Conclusion Générale
- Le modèle "Final" (avec G1/G2) est excellent (R2 > 0.8) et fiable pour le déploiement.
- Le modèle "Précoce" (sans notes) est un indicateur faible mais peut servir à identifier des profils à risque (via `failures` et `absences`) plutôt qu'à prédire une note précise.
- Structure de la soutenance définie.

## Jour 5 - 16/01/2026

### Étapes / Actions réalisées
- **Audit des Modèles** : demande de validation rigoureuse des choix.
- Création d'un protocole de benchmark (`model_benchmarking.ipynb`) : 5-fold CV sur 5 modèles (Linear, RF, LightGBM, SVR, MLP).
- **Résultats du Benchmark** :
    - *Scénario Final* : La Régression Linéaire (R2=0.784) bat Random Forest (0.775) et LightGBM.
    - *Scénario Précoce* : SVR domine (R2=0.16) mais reste faible. Les réseaux de neurones (MLP) échouent (trop peu de données).

### Observations / Difficultés
- Le Deep Learning n'est pas adapté à ce volume de données (< 1000 lignes).
- La simplicité de la régression linéaire la rend très attractive pour le déploiement.

## Jour 6 - 17/01/2026

### Étapes / Actions réalisées
- **Test d'Ensembling** : Hypothèse que combiner Linear + RF pourrait lisser les erreurs.
- Implémentation d'un `VotingRegressor`.
- **Résultat** : Victoire de l'Ensemble avec **R2 = 0.795** et **RMSE = 1.59**.

### Décision Finale
- Le modèle déployé sera l'**Ensemble (Linear Regression + Random Forest)**.
- Il offre le meilleur compromis performance/robustesse.

## Jour 7 - 18/01/2026

### Raffinements Finaux et Déploiement

#### 1. Affinement du Modèle (Gestion des Abandons)
- **Constat** : L'analyse détaillée des données a montré que les notes à 0 (pour `G3`) correspondent majoritairement à des abandons ou absences, faussant la régression pour les élèves actifs.
- **Décision** : Le modèle déployé a été ré-entraîné sur un subset filtré (`G3 > 0`).
- **Impact** : Meilleure précision pour les utilisateurs réels de l'application (étudiants en cours de scolarité).

#### 2. Éthique et UI/UX
- **Data Minimization** : Retrait des champs sensibles (santé, famille) de l'interface de saisie.
- **Valeurs neutres** : Injection transparente de valeurs médianes/neutres dans l'API pour ces champs, afin de ne pas casser le modèle tout en protégeant l'utilisateur.
- **Interface** : Adoption d'une navigation par menu latéral (`sidebar`) pour séparer clairement l'accueil des outils de prédiction.

#### 3. Industrialisation (Traefik, Monitoring & MLOps)
- **Architecture Distante** : Configuration complète de Traefik pour exposer l'API, le Frontend, Prometheus, Grafana, MLFlow et Prefect via des URLs dédiées (`*.dev.brad.team`).
- **Monitoring Observabilité** : Dashboard Grafana connecté à Prometheus pour suivre la santé de l'API (RPS, Latence, Status Up/Down) en temps réel.
- **Experiment Tracking (MLFlow)** : Intégration de MLFlow pour tracer chaque requête API (inputs/outputs) et versionner les modèles lors du ré-entraînement.
- **Orchestration (Prefect)** : Mise en place de Prefect pour automatiser le pipeline de ré-entraînement (`retraining_flow`), assurant reproductibilité et gestion des flux.
- **Optimisation Infrastructure (Debian 13)** : Configuration spécifique de cAdvisor pour supporter `cgroupv2`, garantissant une visibilité complète des métriques conteneurisées sur le serveur de production.

## Choix d'Architecture de Déploiement

Nous avons opté pour une stratégie de **construction locale des images conteneurisées** (CI/CD via SSH + `podman-compose build`) plutôt que l'utilisation d'un registre d'images externe (type Docker Hub ou GHCR).

**Justification :**
1.  **Spécificité de la Solution** : Contrairement à un SaaS standardisé destiné à être dupliqué à l'identique pour des milliers de clients, notre solution est une **application sur-mesure**, adaptée aux données spécifiques de l'établissement.
2.  **Unicité du Déploiement** : Le déploiement est unique (une seule instance de production). Il n'y a pas de nécessité de distribuer des images compressées à travers un cluster complexe.
3.  **Simplicité & Frugalité** : 
    *   Évite la gestion de secrets supplémentaires pour un registre privé.
    *   Réduit la bande passante (pas d'upload/download d'images lourdes, seul le code source léger transite).
    *   Les couches de cache locales de Podman sur le serveur rendent les reconstructions très rapides pour les mises à jour mineures.